In [24]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Collecting xgboost
  Downloading xgboost-1.7.3-py3-none-win_amd64.whl (89.1 MB)
     --------------------------------------- 89.1/89.1 MB 12.8 MB/s eta 0:00:00
Requirement already satisfied: numpy in c:\users\mosac\anaconda3\lib\site-packages (from xgboost) (1.21.5)
Requirement already satisfied: scipy in c:\users\mosac\anaconda3\lib\site-packages (from xgboost) (1.9.1)
Installing collected packages: xgboost
Successfully installed xgboost-1.7.3
Note: you may need to restart the kernel to use updated packages.

In [25]:
%pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
%pip install mediapipe 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from  xgboost import XGBClassifier
import xgboost
from sklearn.metrics import r2_score
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [28]:
def calculate_angles(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #middle
    c = np.array(c) #last
    #Calculates the angle formed by the three dots
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)#Convert to degrees
    #An angle cannot be more than 180
    if angle > 180.0:
        angle = 360- angle

    return angle

In [ ]:
#3 angale
#every anagle creates from:a,b,c
data = {
    'elbow': [],    
    'knee': [],    
    'shoulder': [],
              
    'label': [] #The value of the character in terms of the X-axis
}



Direction of trainer in the image 

Calculate the angels

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            #להביא את הזוויות של הברך,מרפק וכתף
            # Get coordinates
            R_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            R_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            R_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            #Visibility Extraction
            R_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            

            L_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            L_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            L_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            #Visibility Extraction
            L_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            

            # Calculate angle
            if R_V > L_V:
                angle = calculate_angle(R_hip, R_knee, R_ankle)
            else:
                angle = calculate_angle(L_hip, L_knee, L_ankle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(L_knee, [1280, 720]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#run over all the images in images folder
directory = r'C:\Users\Orel\Documents\GitHub\FitnessAI\images'
label = ''
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
    
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            #run over the images inside the folders
            #takes every image and put it inside a frame,f=image
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)#read the image to frame of cv2

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#paint the pose
            image.flags.writeable = False
      
            results = pose.process(image)#the picther with the scale and dots

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#change the color format

            #Extract  the dots from the image,every dots contain x,y,z,visabillity and saved in arr
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass

            #drew the dots
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), #different color-blue
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) #different color-pink
                            )
         #binary classification between d,s,b
         #ask in the folder give label by the name written in the image
        if 'deadlift' in f:
            label = 'deadlift'
        elif 'squat' in f:
            label = 'squat'
        elif 'bench' in f:
            label = 'bench'
        else:
            continue
        
        
       
        #add those angels values to the data array.
        for i in range(11,len(mp_pose.PoseLandmark)-10):#run over all the lendmark of one imag
            data["elbow"].append(landmarks[i].elbow)#add the landmark

        for i in range(11,len(mp_pose.PoseLandmark)-10):
            data["knee"].append(landmarks[i].knee)
           
        for i in range(11,len(mp_pose.PoseLandmark)-10):
            data["shoulder"].append(landmarks[i].shoulder) 

        for i in range(11,len(mp_pose.PoseLandmark)-10):
            data["label"].append(landmarks[i].label)       

            


In [ ]:
#calculate the angle for each position 
elbowF=landmarks[angle];
elbowM=landmarks[angle];
elbowL=landmarks[angle];

kneeF=landmarks[angle];
kneeM=landmarks[angle];
kneeL=landmarks[angle];

shoulderF=landmarks[angle];
shoulderM=landmarks[angle];
shoulderL=landmarks[angle];



In [ ]:
#Make data array into pandas DataFrame
df = pd.DataFrame(data=data,columns=data.keys())
df_test = pd.DataFrame(data=test_data,columns=test_data.keys())